In [35]:
from Tonnetz_Select import fromMidiToPCS as fmpc
from Tonnetz_Select import analysisFromCorpus
from TrajectoryCalculationsWithClass import NewTrajectory, TrajectoryLookBefore
from graph_creation import CreateGraph
from FirstNotePosition import PlaceFirstNote
from structural_functions import mergeDicts
import os
import pickle
from itertools import islice, groupby
from operator import itemgetter
import music21 as ms
import pretty_midi as ptm

In [36]:
def pickleSave(dictOfGraphs, Composer_Name) :
    completeName = "Comparison_Results/Nottingham_Dataset/"+ Composer_Name + ".p"
    with open(completeName, 'wb') as config_dictionary_file:
      # Step 3
      pickle.dump(dictOfGraphs, config_dictionary_file)

In [57]:
def findInstruments(file):
    instruments = []
    time_signatures = []
    s = ms.converter.parse(file)
    for element in s.recurse() :
        if 'TimeSignature' in element.classes :
            time_signatures.append(element.ratioString)
        if 'Instrument' in element.classes :
            x = str(element)
            if x != '' :
                if ': ' in x :
                    newstring = x.split(': ', -1)[1]
                    instruments.append(newstring)
                else :
                    instruments.append(x)
    time_signatures = list(map(itemgetter(0), groupby(time_signatures)))
    return instruments, time_signatures

In [38]:
def TempoAdd(directory):
    s = ptm.PrettyMIDI(directory)
    tempo = s.estimate_tempo()
    return int(tempo)

In [39]:
def GraphOfNewPiece(newPiece, directory):

    # extract the directory and the file extension from the piece
    shortfilename = os.path.splitext(os.path.basename(newPiece))[0]
    # Choose from where to parse (directory, corpus)
    if directory == 'corpus':
        file = ms.corpus.parse(newPiece)
        chordList, Tonnetz = analysisFromCorpus(file)
    # add the instruments
        instruments, time_signatures = findInstruments(newPiece)
        tempo = []
    else:
        if newPiece.endswith(".mid") or newPiece.endswith(".MID"):
            complete_name = directory + '/' + newPiece
            chordList, Tonnetz = fmpc(complete_name)
        # add the instruments
            instruments, time_signatures = findInstruments(complete_name)
            tempo = TempoAdd(complete_name)
    firstPoint = PlaceFirstNote(chordList, Tonnetz)
    # build 2 different trajectories
    trajectoryFut = NewTrajectory(chordList, Tonnetz, firstPoint)
    trajectoryFut.addType('Trajectory With Future')
    trajectoryFut.addInstruments(instruments)
    trajectoryFut.addTime_signature_changes(time_signatures)
    trajectoryFut.addTempo(tempo)
    graphFut = CreateGraph(trajectoryFut)
    graphFut.addName(shortfilename)
    
    trajectoryRec = TrajectoryLookBefore(chordList, Tonnetz, firstPoint)
    trajectoryRec.addType('Recursive Trajectory')
    trajectoryRec.addInstruments(instruments)
    trajectoryRec.addTime_signature_changes(time_signatures)
    trajectoryRec.addTempo(tempo)
    graphRec = CreateGraph(trajectoryRec)
    graphRec.addName(shortfilename)
    
    return graphFut, graphRec

In [ ]:
def inputQuestions() : 
    composer = input('Do you know the Composer of this Directory ? ')
    style = input('Do you know the Style of the works in this Directory ? ')
    harmony = input('Do you know the Harmony Style of the works in this Directory ? ')
    return composer, style, harmony

In [ ]:
def addingLabels(composer, style, harmony, graph):
    if composer != 'No' :
        graph.addComposer(composer)
    if style != 'No' :
        graph.addStyle(style)
    if harmony != 'No':
        graph.addHarmonyStyle(harmony)

In [ ]:
def GetWorksByDirectory(directory, composer, style):
    listOfGraphs = []
#     composer, style, harmony = inputQuestions()
    for file in os.listdir(directory):
        if file.endswith(".mid") or file.endswith(
                ".MID") or file.endswith(".mxl") or file.endswith(".xml"):
            try:
                print("Building Trajectory for ", file)
                graphFut, graphRec = GraphOfNewPiece(file, directory)
#                 addingLabels(composer, style, harmony, graph)
                graphFut.addStyle(style)
                graphFut.addComposer(composer)
                graphRec.addStyle(style)
                graphRec.addComposer(composer)
                listOfGraphs.append([graphFut, graphRec])
            except BaseException:
                print("--> Cannot build Trajectory for ", file)
    return listOfGraphs

In [ ]:
def GetWorksByComposer(composerName, composer, style):
    listOfGraphs = []
#     composer, style, harmony = inputQuestions()
    listofWorks = ms.corpus.getComposer(composerName)
    if len(listofWorks) > 60 :
        listofWorks = list(islice(listofWorks, 60))
    if len(listofWorks) > 0:
        for piece in listofWorks:
            try:
                print("Building Trajectory for ", piece)
                graphFut, graphRec = GraphOfNewPiece(piece, 'corpus')
#                 addingLabels(composer, style, harmony, graph)
                graphFut.addStyle(style)
                graphFut.addComposer(composer)
                graphRec.addStyle(style)
                graphRec.addComposer(composer)
                listOfGraphs.append([graphFut, graphRec])
            except BaseException:
                print("--> Cannot build Trajectory for ", piece)
    return listOfGraphs

In [ ]:
beethoven = GetWorksByDirectory('Midi_files/Classical/Beethoven', 'beethoven', 'classical')
beethovenCorpus = GetWorksByComposer('beethoven', 'beethoven', 'classical')

In [ ]:
chopin = GetWorksByDirectory('Midi_files/Classical/Chopin', 'chopin', 'classical')
chopinCorpus = GetWorksByComposer('chopin', 'chopin', 'classical')

In [ ]:
mozart = GetWorksByDirectory('Midi_files/Classical/Mozart', 'mozart', 'classical')
mozartCorpus = GetWorksByComposer('mozart', 'mozart', 'classical')

In [ ]:
schumann = GetWorksByDirectory('Midi_files/Classical/Schumann', 'schumann', 'classical')
schumannCorpus = GetWorksByComposer('schumann', 'schumann', 'classical')

In [ ]:
palestrina = GetWorksByComposer('palestrina', 'palestrina', 'renaissance')
monteverdi = GetWorksByComposer('monteverdi', 'monteverdi', 'renaissance')

In [ ]:
bach = GetWorksByComposer('bach', 'bach', 'barok')

In [ ]:
jazz = GetWorksByDirectory('Midi_files/Standard_Jazz/Random', 'Unknown', 'jazz')

In [ ]:
pickleSave(chopin, 'chopin')
pickleSave(chopinCorpus, 'chopinCorpus')

pickleSave(beethoven, 'beethoven')
pickleSave(beethovenCorpus, 'beethovenCorpus')

pickleSave(schumann, 'schumann')
pickleSave(schumannCorpus, 'schumannCorpus')

pickleSave(mozart, 'mozart')
pickleSave(mozartCorpus, 'mozartCorpus')

pickleSave(palestrina, 'palestrina')

pickleSave(monteverdi, 'monteverdi')

pickleSave(bach, 'bach')

# pickleSave(jazz, 'jazz')

In [27]:
def startWithStyle(file):
    if file.startswith('jigs') :
        style = 'jigs'
    elif file.startswith('hpps') :
        style = 'hpps'
    elif file.startswith('ashover') :
        style = 'ashover'
    elif file.startswith('playford') :
        style = 'playford'
    elif file.startswith('reels'):
        style = 'reels'
    elif file.startswith('waltzes'):
        style = 'waltz'
    else :
        style = 'xmas'
    return style

In [28]:
def GetWorksByNottingham(directory):
    listOfGraphs = []
#     composer, style, harmony = inputQuestions()
    for file in os.listdir(directory):
        if file.endswith(".mid") or file.endswith(
                ".MID") or file.endswith(".mxl") or file.endswith(".xml"):
            style = startWithStyle(file)
            try:
                print("Building Trajectory for ", file)
                graphFut, graphRec = GraphOfNewPiece(file, directory)
#                 addingLabels(composer, style, harmony, graph)
                graphFut.addStyle(style)
#                 graphFut.addComposer(composer)
                graphRec.addStyle(style)
#                 graphRec.addComposer(composer)
                listOfGraphs.append([graphFut, graphRec])
            except BaseException:
                print("--> Cannot build Trajectory for ", file)
    return listOfGraphs